In [ ]:
import ee, os
import geemap
import numpy as np
import pandas as pd
import spyndex

In [ ]:
ee.Initialize()
Map = geemap.Map()
roi = geemap.shp_to_ee(r"E:\研究区土地利用\研究区\区域\TPB.shp").geometry()
elevation = ee.Image("USGS/SRTMGL1_003").clip(roi)
# 计算坡度和坡向
terrain = ee.Terrain.products(elevation)
# 提取坡度和坡向
slope = terrain.select('slope')
aspect = terrain.select('aspect')

In [ ]:
def masksr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    #thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True)\
        .updateMask(qaMask)\
        .updateMask(saturationMask)

# modis 去云函数
def mask_clouds(image):
    # 选择QA波段
    QA = image.select('state_1km')
    
    # 创建掩膜位掩盖不同的干扰
    internal_cloud_flag = 1 << 10  # 内部云算法标志
    cloud_shadow_flag = 1 << 2  # 云阴影标志
    snow_ice_flag = 1 << 11  # 雪/冰覆盖标志
    
    # 返回同时屏蔽云、云阴影和雪/冰区域的图像
    mask = QA.bitwiseAnd(internal_cloud_flag).eq(0) \
           .And(QA.bitwiseAnd(cloud_shadow_flag).eq(0)) \
           .And(QA.bitwiseAnd(snow_ice_flag).eq(0))
    
    return image.updateMask(mask)


def hillshade(img, elevation):
  # 太阳方位角
  azimuth = img.getNumber('SUN_AZIMUTH')
  # 太阳天顶角=90-太阳仰角
  zenith = ee.Number(90).subtract(img.getNumber('SUN_ELEVATION'))
  # 计算地形阴影
  shaded_relief = ee.Terrain.hillShadow(elevation, azimuth, zenith, 200, True)
  return img.updateMask(shaded_relief)

# landsat8传感器校正
def correction(landsat8):
    SR_B2 = landsat8.select('SR_B2').multiply(0.8850).add(0.0183).rename('SR_B2').toFloat()
    SR_B3 = landsat8.select('SR_B3').multiply(0.9317).add(0.0123).rename('SR_B3').toFloat()
    SR_B4 = landsat8.select('SR_B4').multiply(0.9372).add(0.0123).rename('SR_B4').toFloat()
    SR_B5 = landsat8.select('SR_B5').multiply(0.8339).add(0.0448).rename('SR_B5').toFloat()
    SR_B6 = landsat8.select('SR_B6').multiply(0.8639).add(0.0306).rename('SR_B6').toFloat()
    SR_B7 = landsat8.select('SR_B7').multiply(0.9165).add(0.0116).rename('SR_B7').toFloat()
    QA_PIXEL = landsat8.select('QA_PIXEL')
    QA_RADSAT = landsat8.select('QA_RADSAT')
    image = SR_B2.addBands([SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,QA_PIXEL,QA_RADSAT])
    image = image.copyProperties(landsat8, ['SUN_AZIMUTH', 'SUN_ELEVATION'])
    return image

In [ ]:
def get_landsat_data(year):
    startdate = str(year) + '-4-01'
    enddate = str(year) + '-9-30'
    #数据集
    l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                .filterBounds(roi)\
                .filterDate(startdate, enddate)\
                .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT'])\
                .map(correction)
    
    l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
                .filterBounds(roi)\
                .filterDate(startdate, enddate)\
                .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT'],
                        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT']) 
                        
    l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
                .filterBounds(roi)\
                .filterDate(startdate, enddate)\
                .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT'],
                        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT']) 
                        
    merge_image = ee.ImageCollection(l8.merge(l7).merge(l5))\
                                        .map(masksr)\
                                        .map(lambda image:hillshade(image, elevation))\
                                        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
        
    return merge_image

def get_modis_data(year):
    startdate = str(year) + '-4-01'
    enddate = str(year) + '-9-30'

    modis = ee.ImageCollection('MODIS/061/MOD09GA')\
                        .filterBounds(roi)\
                        .filterDate(startdate, enddate)\
                        .map(mask_clouds)\
                        .select(['sur_refl_b03','sur_refl_b04','sur_refl_b01','sur_refl_b02','sur_refl_b06','sur_refl_b07'],
                                ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])\
                        .map(lambda image:image.multiply(0.0001))
    return modis


In [ ]:
def calculation(collection):
    N = collection.select(['SR_B5'])
    B = collection.select(['SR_B2'])
    G = collection.select(['SR_B3'])
    R = collection.select(['SR_B4'])
    S1 = collection.select(['SR_B6'])
    S2 = collection.select(['SR_B7'])
    
    # 光谱指标
    bndvi = spyndex.computeIndex(index = ["BNDVI"],params = {"N": N,"B": B}).rename('bndvi')
    cig = spyndex.computeIndex(index = ['CIG'],params = {"N": N,'G': G}).rename('cig')
    cvi = spyndex.computeIndex(index = ['CVI'],params = {"N": N,'R': R,'G': G}).rename('cvi')
    dsi = spyndex.computeIndex(index = ['DSI'],params = {'S1': S1,'N': N}).rename('dsi')
    dbsi = spyndex.computeIndex(index = ['DBSI'],params = {'S1': S1,'G': G,'N': N,'R': R}).rename('dbsi')
    dvi = spyndex.computeIndex(index = ['DVI'], params={'N': N,'R': R}).rename('dvi')
    endvi = spyndex.computeIndex(index = ['ENDVI'],params={'N': N,'G': G,"B": B}).rename('endvi')
    evi = spyndex.computeIndex(index=['EVI'],params={'g':2.5,'N': N,'R': R,'C1': 6.0,'C2': 7.5,"B": B,'L': 1.0}).rename('evi')
    evi2 = spyndex.computeIndex(index=['EVI2'],params={'g':2.5,'N': N,'R': R,'L': 1.0}).rename('evi2')
    gbndvi = spyndex.computeIndex(index=['GBNDVI'],params={'N': N,'G': G,"B": B}).rename('gbndvi')
    gndvi = spyndex.computeIndex(index=['GNDVI'],params={'N': N,'G': G}).rename('gndvi')
    gosavi = spyndex.computeIndex(index=['GOSAVI'],params={'N': N,'G': G}).rename('gosavi')
    grndvi = spyndex.computeIndex(index=['GRNDVI'],params={'N': N,'G': G,'R': R}).rename('grndvi')
    grvi = spyndex.computeIndex(index=['GRVI'],params={'N': N,'G': G}).rename('grvi')
    ipvi = spyndex.computeIndex(index=['IPVI'],params={'N': N,'R': R}).rename('ipvi')
    mgrvi = spyndex.computeIndex(index=['MGRVI'],params={'G': G,'R': R}).rename('mgrvi')
    mndvi = spyndex.computeIndex(index=['MNDVI'],params={'N': N,'S2': S2}).rename('mndvi')
    mnli = spyndex.computeIndex(index=['MNLI'],params={'L': 0.5,'N': N,'R': R}).rename('mnli')
    msavi = spyndex.computeIndex(index=['MSAVI'],params={'N': N,'R': R}).rename('msavi')
    msr = spyndex.computeIndex(index=['MSR'],params={'N': N,'R': R}).rename('msr')
    ndpi = spyndex.computeIndex(index=['NDPI'],params={'N': N,'alpha':0.74,'R': R,'S1': S1}).rename('ndpi')
    ndvi = spyndex.computeIndex(index=['NDVI'],params={'N': N,'R': R}).rename('ndvi')
    ngrdi = spyndex.computeIndex(index=['NGRDI'],params={'G': G,'R': R}).rename('ngrdi')
    nirv = spyndex.computeIndex(index=['NIRv'],params={'N': N,'R': R}).rename('nirv')
    nli = spyndex.computeIndex(index=['NLI'],params={'N': N,'R': R}).rename('nli')
    osavi = spyndex.computeIndex(index=['OSAVI'],params={'N': N,'R': R}).rename('osavi')
    rdvi = spyndex.computeIndex(index=['RDVI'],params={'N': N,'R': R}).rename('rdvi')
    sarvi = spyndex.computeIndex(index=['SARVI'],params={'L': 1.0,'N': N,'R': R,"B": B}).rename('sarvi')
    savi = spyndex.computeIndex(index=['SAVI'],params={'L': 1.0,'N': N,'R': R}).rename('savi')
    sr = spyndex.computeIndex(index=['SR'],params={'N': N,'R': R}).rename('sr')
    tvi = spyndex.computeIndex(index=['TVI'],params={'N': N,'R': R}).rename('tvi')
    wdrvi = spyndex.computeIndex(index=['WDRVI'],params={'alpha':0.2,'N': N,'R': R}).rename('wdrvi')
    ndwi = spyndex.computeIndex(index=['NDWI'],params={'G': G,'N': N}).rename('ndwi')
    bi = spyndex.computeIndex(index=['BI'],params={'S1': S1,'N': N,'R': R,"B": B}).rename('bi')
    mtvi1 = spyndex.computeIndex(index=['MTVI1'],params={'N': N,'R': R,'G': G}).rename('mtvi1')
    mtvi2 = spyndex.computeIndex(index=['MTVI2'],params={'N': N,'R': R,'G': G}).rename('mtvi2')

    list1 = [cig, cvi, dsi, dbsi, dvi, endvi, evi, evi2, gbndvi,\
               gndvi, gosavi, grndvi, grvi, ipvi, mgrvi, mndvi, mnli, msavi,\
               msr, ndpi, ndvi, ngrdi, nirv, nli, osavi, rdvi, sarvi, savi,sr,\
               tvi, wdrvi, ndwi, bi, mtvi1, mtvi2, N, B, G, R, S1, S2]
    total_image = bndvi.addBands(list1)
    return total_image

In [ ]:
df = pd.read_csv(r"E:\赖学长\那曲市\data\样本点数据.csv")
df.drop(['Elevation_','slope'],axis=1)
lt = df['date'].to_list()
lt_re = [int(str(x)[:4]) for x in lt]
df['date'] = lt_re

In [ ]:
for year in range(2000, 2023):
    landsat = get_landsat_data(year)\
                .map(calculation)\
                .max()\
                .clip(roi)
    
    modis = get_modis_data(year)\
                .map(calculation)\
                .max()\
                .clip(roi)
    
    result = landsat.unmask(modis).addBands([elevation, slope, aspect])
    
    if year in list(df['date']):
        dt = df[df['date']==year]
        points = geemap.df_to_ee(dt, latitude='latitude', longitude='longitude')

        outfile = r"E:\赖学长\那曲市\年份"+os.sep+str(year)+".csv"
        final_Collection = geemap.extract_values_to_points(points, result, outfile, scale=30,crs='epsg:4326',tileScale=16)
